In [ ]:
from src.models.components.superpoint import SuperPoint
from linetr.line_detector import LSD
from linetr.line_process import preprocess, line_tokenizer
from linetr_utils.util_lines import find_line_matches, calculate_line_overlaps, conv_fixed_size
import torch
import tqdm

import os
import os.path as osp
import argparse
from datetime import date
import json
import random
import time
from pathlib import Path
import numpy as np
import numpy.linalg as LA
import torch.linalg
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
import csv

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader


import pandas as pd
import glm

In [ ]:
conf = {
    "data": {
        "name": "gcl_synth",
        "image_path": "./assets/dataset/raw_images",
        "output_path": "./assets/dataset/dataset_h5",
        "image_type": "*.jpg",
        "resize": (640, 480),
        "visualize": True,
        "n_iters": 1,
        "choose_worker": 0,
        "nWorkers": 1
    },
    "augmentation": {
        "num": 1,
        "photometric": {
            "enable": True,
            "primitives": [
                "random_brightness", "random_contrast", "additive_speckle_noise",
                "additive_gaussian_noise", "additive_shade", "motion_blur"
            ],
            "params": {
                "random_brightness": {"max_abs_change": 50},
                "random_contrast": {"strength_range": [0.3, 1.5]},
                "additive_gaussian_noise": {"stddev_range": [0, 10]},
                "additive_speckle_noise": {"prob_range": [0, 0.0035]},
                "additive_shade": {
                    "transparency_range": [-0.5, 0.5],
                    "kernel_size_range": [100, 150]
                },
                "motion_blur": {"max_kernel_size": 3}
            }
        },
        "homographic": {
            "enable": True,
            "params": {
                "perspective": True,
                "scaling": True,
                "translation": True,
                "rotation": True,
                "patch_ratio": 0.85,
                "perspective_amplitude_x": 0.2,
                "perspective_amplitude_y": 0.2,
                "scaling_amplitude": 0.2,
                "max_angle": 1.0472,
                "allow_artifacts": True
            },
            "valid_border_margin": 3
        }
    },
    "feature": {
        "linetr": {
            "min_length": 16,
            "token_distance": 8,
            "max_tokens": 21,
            "remove_borders": 1,
            "max_sublines": 250,
            "thred_reprojected": 3,
            "thred_angdiff": 2,
            "min_overlap_ratio": 0.3
        },
        "superpoint": {
            "nms_radius": 4,
            "keypoint_threshold": 0.005,
            "remove_borders": 4,
            "max_keypoints": 256
        }
    }
}


In [ ]:
device = torch.device('cuda')
model = SuperPoint(conf['feature']['superpoint']).to(device).eval()
linedetector = LSD(conf['feature']['linetr'])

In [ ]:
path = "/home/kmuvcl/source/oldCuTi/CuTi/matterport/mp3d_planercnn_json/cached_set_moon_val_vp.json"
root = "/home/kmuvcl/source/oldCuTi/CuTi/matterport/"

In [ ]:
with open(path) as file:
    split = json.load(file)

In [ ]:
len(split['data'])

In [ ]:
split['data']['0'].keys()

In [ ]:
for i in tqdm(range(len(split['data']))):
    del split['data'][str(i)]['gt_corrs']
    del split['data'][str(i)]['0']['annotations']
    del split['data'][str(i)]['1']['annotations']

In [ ]:
images_list = []
lines_list = []
vps_list = []
poses_list = []
intrinsics_list = []

original_basepath = "/Pool1/users/jinlinyi/dataset/mp3d_rpnet_v4_sep20"
data_path = "/home/kmuvcl/source/oldCuTi/CuTi/matterport"

resize = (640,480)

In [ ]:
def read_data(path, device, resize):
    image = cv2.imread(str(path), cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    if resize[0] != -1:
        image = cv2.resize(image.astype('float32'), (resize[0], resize[1]))
        gray = cv2.resize(gray.astype('uint8'), (resize[0], resize[1]))
    gray_torch = torch.from_numpy(gray/255.).float()[None, None].to(device)
    
    return image, gray, gray_torch

def read_line_file(filename: str, min_line_length=10):
        segs = []  # line segments

        with open(filename, "r") as csvfile:
            csvreader = csv.reader(csvfile)
            for row in csvreader:
                segs.append([float(row[0]), float(row[1]), float(row[2]), float(row[3])])
        segs = np.array(segs, dtype=np.float32)
        lengths = LA.norm(segs[:, 2:] - segs[:, :2], axis=1)
        segs = segs[lengths > min_line_length]
        return segs
    

In [ ]:
klines_dict = {}

In [ ]:
split['data'].values()

In [ ]:
for i, data in enumerate(tqdm(split["data"].values())):
    vps = []
    images_list = []
    
    for img_idx in ["0", "1"]:
        img_path = data[img_idx]["file_name"].replace(original_basepath, data_path)
        images_list.append(img_path)
    images = []
    grays = []
    for j in range(2):
        images.append(cv2.imread(images_list[j], cv2.IMREAD_COLOR))
        
    images[0] = cv2.resize(images[0].astype('float32'), (480,640))
    images[1] = cv2.resize(images[1].astype('float32'), (480,640))
    grays.append(cv2.cvtColor(images[0], cv2.COLOR_RGB2GRAY))
    grays.append(cv2.cvtColor(images[1], cv2.COLOR_RGB2GRAY))
    
    grays[0] = cv2.resize(grays[0].astype('uint8'), (resize[0], resize[1]))
    grays[1] = cv2.resize(grays[1].astype('uint8'), (resize[0], resize[1]))
    
    image0_torch = torch.from_numpy(grays[0]/255.).float()[None, None].to(device)
    image1_torch = torch.from_numpy(grays[1]/255.).float()[None, None].to(device)
    
    height, width = image_shape = grays[0].shape
    
    with torch.no_grad():
        pred0 = model({'image': image0_torch})
        pred1 = model({'image': image1_torch})
        
    valid_mask0 = np.ones_like(grays[0])
    valid_mask1 = np.ones_like(grays[1])  
    
    klns0_cv = linedetector.detect(grays[0])
    klns1_cv = linedetector.detect(grays[1])
    
    try:
        klines0 = preprocess(klns0_cv, image_shape, pred0, mask=valid_mask0, conf=conf['feature']['linetr'])   ## TODO: torch vs. np. 잘 정리하기, tokenizer 다시 정리
        klines1 = preprocess(klns1_cv, image_shape, pred1, mask=valid_mask1, conf=conf['feature']['linetr'])
    except:
        print("line preprocess break")
        break
    klines0 = conv_fixed_size(klines0, conf, func_token=line_tokenizer, pred_sp=pred0)
    klines1 = conv_fixed_size(klines1, conf, func_token=line_tokenizer, pred_sp=pred1)
    
    klns0 = klines0['sublines'].reshape(-1, 2, 2).cpu().numpy()
    klns1 = klines1['sublines'].reshape(-1, 2, 2).cpu().numpy()
    
    keys_l = ['klines', 'sublines', 'angle_sublines','pnt_sublines', 'desc_sublines', \
            'score_sublines', 'resp_sublines', 'mask_sublines', 'num_klns', 'mat_klines2sublines', 'num_slns']
        # klines, resp, angle, pnt, desc, score, mask
    klines0 = {k:v[0] for k,v in klines0.items() if k in keys_l}
    klines1 = {k:v[0] for k,v in klines1.items() if k in keys_l}

    data['0']['klines'] = klines0
    data['1']['klines'] = klines1

    if i ==7:
        break




In [ ]:
klines_dict.keys()

In [ ]:
split['data']['0']['1']['klines']

In [ ]:
with open('cached_set_sp_moon_val_vp.json', 'w') as f : 
	json.dump(split, f, indent=4)

In [ ]:
len(klns0_cv)

In [ ]:
klines0.keys()

In [ ]:
klines0['num_slns0']

In [ ]:
import h5py
import torch
import numpy as np
import math

In [ ]:
def load_h5py_to_dict(file_path):
    with h5py.File(file_path, 'r') as f:
        return {key: torch.tensor(f[key][:]) for key in f.keys()}

In [ ]:
dict = load_h5py_to_dict('/home/kmuvcl/source/oldCuTi/CuTi/matterport/rgb/2t7WUuJeko7/0_2_11_sp_line.h5py')

In [ ]:
dict.keys()

In [ ]:
dict['sublines']

In [ ]:
dict['mask_sublines'][0][4]

In [ ]:
dict['sublines']

In [ ]:
dict['klines'][0,247]

In [ ]:
sublines = dict['sublines'][0]

In [ ]:
dict['desc_sublines'].shape

In [ ]:
def normalize_safe_np(v, axis=-1, eps=1e-6):
    de = LA.norm(v, axis=axis, keepdims=True)
    de = np.maximum(de, eps)
    return v / de

def segs2lines_np(segs):
    ones = np.ones(len(segs))
    ones = np.expand_dims(ones, axis=-1)
    p1 = np.concatenate([segs[:, :2], ones], axis=-1)
    p2 = np.concatenate([segs[:, 2:], ones], axis=-1)
    lines = np.cross(p1, p2)
    return normalize_safe_np(lines)

def normalize_segs(self, lines, pp, rho=517.97):
    pp = np.array([pp[0], pp[1], pp[0], pp[1]], dtype=np.float32)
    return (lines - pp)/rho

def sample_segs_np(segs, num_sample):
    num_segs = len(segs)
    sampled_segs = np.zeros([num_sample, 4], dtype=np.float32)
    mask = np.zeros([num_sample, 1], dtype=np.float32)
    if num_sample > num_segs:
        sampled_segs[:num_segs] = segs
        mask[:num_segs] = np.ones([num_segs, 1], dtype=np.float32)
    else:
        lengths = LA.norm(segs[:, 2:] - segs[:, :2], axis=-1)
        prob = lengths / np.sum(lengths)
        idxs = np.random.choice(segs.shape[0], num_sample, replace=True, p=prob)
        sampled_segs = segs[idxs]
        mask = np.ones([num_sample, 1], dtype=np.float32)
    return sampled_segs, mask

def coordinate_yup(segs, org_h):
    H = np.array([0, org_h, 0, org_h])
    segs[:, 1] = -segs[:, 1]
    segs[:, 3] = -segs[:, 3]
    return (H + segs)

def get_line_dist(line):
    sp = line[0]
    ep = line[1]
    return np.sqrt(np.sum((ep-sp)**2))

def point_on_line(line, dist_px):
    assert dist_px >= 0, 'distance should be positive!'
    assert get_line_dist(line) >= dist_px, 'distance should be smaller than line length!'

    sp,ep = line
    vec = ep - sp
    if vec[0] != 0:
        m = vec[1]/vec[0]
        x = np.sqrt(dist_px**2/(1+m**2))
        y = m*x
    else:
        x = 0
        y = dist_px if ep[1]-sp[1] > 0 else -dist_px

    return (x,y)+sp

def sample_descriptors(keypoints, descriptors, s: int = 8):
    """ Interpolate descriptors at keypoint locations """
    b, c, h, w = descriptors.shape
    keypoints = keypoints - s / 2 + 0.5
    keypoints /= torch.tensor([(w*s - s/2 - 0.5), (h*s - s/2 - 0.5)],
                              ).to(keypoints)[None]
    keypoints = keypoints*2 - 1  # normalize to (-1, 1)
    args = {'align_corners': True} if int(torch.__version__[2]) > 2 else {}
    descriptors = torch.nn.functional.grid_sample(
        descriptors, keypoints.view(b, 1, -1, 2), mode='bilinear', **args)
    descriptors = torch.nn.functional.normalize(
        descriptors.reshape(b, c, -1), p=2, dim=1)
    return descriptors


In [ ]:
sublines.shape

In [ ]:
sublines 

In [ ]:
sublines = sublines.reshape(250,-1).numpy()

In [ ]:
sublines

In [ ]:
sublines = coordinate_yup(sublines,480)

In [ ]:
sublines

In [ ]:
tokens = []
token_distance = 8
width = 640
height = 480
max_tokens = 21
slines_all, num_slines_all = [], []
tokens_all, masks_all = [], []
response_all, angle_all = [], []

In [ ]:
kline = dict['klines']
klength = dict['length_klines']

In [ ]:
kline[0][0]

In [ ]:
num_tokens = int(math.ceil(klength[0][0] / 8))

In [ ]:
num_tokens

In [ ]:
for i, (kline, klength) in enumerate(zip(dict['klines'][0].numpy(), dict['length_klines'][0].numpy())):
    print(kline)
    tokens = []
    num_tokens = int(math.ceil(klength / token_distance))
    print(num_tokens)
    for i_token in range(num_tokens-1):
        dist = i_token * token_distance
        token = point_on_line(kline, dist)
        tokens.append(token)
    token = kline[1]
    token[0]=token[0].clip(max=width-0.6)
    token[1]=token[1].clip(max=height-0.6)
    tokens.append(token)
    sp,ep = kline
    num_sublines = int(math.ceil(num_tokens / max_tokens))
    sublines = np.zeros((num_sublines,2,2))
    sublines[0,0] = sp
    sublines[-1,1] = ep
    for i_sline in range(num_sublines-1):
        mid_token = tokens[(i_sline+1)*max_tokens-1]
        sublines[i_sline, 1] = mid_token
        sublines[i_sline+1, 0] = mid_token
    slines_all.extend(sublines)
    num_slines_all.append(num_sublines)

    # line tokens & masks for sublines
    tokens_sline = np.zeros((num_sublines,max_tokens,2))
    masks_sline = np.zeros((num_sublines,max_tokens+1,1))
    masks_sline[:,0] = 1

    for i_sline in range(num_sublines):
        i_start = i_sline*max_tokens
        i_end = i_start + max_tokens
        tmp_tokens = np.asarray(tokens[i_start:i_end])
        tokens_sline[i_sline,:len(tmp_tokens)] = tmp_tokens
        masks_sline[i_sline,1:len(tmp_tokens)+1] = 1
    tokens_all.extend(tokens_sline)
    masks_all.extend(masks_sline)

    resp_sline = np.zeros((num_sublines,1))
    ang_sline = np.zeros((num_sublines,2))
    max_length = token_distance * max_tokens
    for i_sline in range(num_sublines):
        resp_sline[i_sline] = get_line_dist(sublines[i_sline]) / max_length
        ang_sline[i_sline] = dict['angles'][0][i]
    response_all.extend(resp_sline)
    angle_all.extend(ang_sline)
    
    device = 'cpu'
    # device = 'cpu'
    slines = torch.from_numpy(np.vstack(slines_all).reshape(-1,2,2)).float().to(device)
    tokens = torch.from_numpy(np.vstack(tokens_all).reshape(-1,max_tokens,2)).float().to(device)
    masks = torch.from_numpy(np.vstack(masks_all).reshape(-1,max_tokens+1,1)).float().to(device)
    responses = torch.from_numpy(np.vstack(response_all).reshape(-1,1)).float().to(device)
    angles = torch.from_numpy(np.vstack(angle_all).reshape(-1,2)).float().to(device)
    
    # adjacency matrix (mat_keylines2sublines)
    klines2slines = torch.zeros((len(dict['klines'][0]), len(slines))).to(device)
    st_idx = 0
    for i, num_sline in enumerate(num_slines_all):
        klines2slines[i,st_idx:st_idx+num_sline] = 1/num_sline
        st_idx += num_sline

    # descriptors & scores for line tokens
    dense_descriptor = dict['dense_descriptor'][0]
    descriptors = sample_descriptors(tokens[None], dense_descriptor, 8)[0].reshape(256,tokens.shape[0],tokens.shape[1])
    descriptors = descriptors.permute(1,2,0)

    dense_score = dict['dense_score'].transpose(1,2)
    pos_score = [torch.round(tokens).long().reshape(-1,2)]
    pos_score[0][:,0] = pos_score[0][:,0].clip(max=dense_score.shape[1]-1)
    pos_score[0][:,1] = pos_score[0][:,1].clip(max=dense_score.shape[2]-1)
    scores = [s[tuple(k.t())] for s, k in zip(dense_score, pos_score)]
    scores = scores[0].reshape(len(slines), max_tokens, 1)
    break

In [ ]:
slines[None]

In [ ]:
import torch

In [ ]:
vp = torch.tensor([[ 8.8663e-01, -8.8244e-02,  4.5397e-01],
         [ 0.0000e+00,  9.8163e-01,  1.9081e-01],
         [-4.6247e-01, -1.6918e-01,  8.7034e-01]]

In [ ]:
vp = vp.transpose(-2,-1)

In [ ]:
vp

In [ ]:
vp[0]

In [ ]:
torch.dot(vp[0],vp[1])

In [1]:
import numpy as np

In [4]:
with np.load("/home/kmuvcl/source/CuTi/intrinsics.npz") as npz:
    np = npz
    scene0000_00 = np['scene0000_00']
    scene0001_00 = np['scene0001_00']

In [5]:
scene0000_00

array([577.5907 ,   0.     , 318.90543,   0.     , 578.7298 , 242.68361,
         0.     ,   0.     ,   1.     ])

In [6]:
scene0000_01

NameError: name 'scene0000_01' is not defined

In [7]:
scene0001_00

array([577.8706,   0.    , 319.5   ,   0.    , 577.8706, 239.5   ,
         0.    ,   0.    ,   1.    ])

In [8]:
array = np.files

In [9]:
array.sort()

In [10]:
scene0000_00 = np['scene0000_00']

AttributeError: 'NoneType' object has no attribute 'open'

In [12]:
array[1512]

'scene0706_00'

In [2]:
with np.load("/home/kmuvcl/dataset/data/scannet-vp/scene0000_00/frame-000000-vanish.npz") as npz:
        vpts = np.array([npz[d] for d in ["x", "y", "z"]])

In [3]:
vpts

array([[ 7.94929364e-01, -2.46233042e-01,  5.54487317e-01],
       [-6.06701116e-01, -3.23240044e-01,  7.26242608e-01],
       [-4.07651387e-04,  9.13720846e-01,  4.06343490e-01]])

In [4]:
import cv2

In [6]:
image = cv2.imread("/home/kmuvcl/dataset/data/scannet-vp/scene0000_00/frame-000000-color.png")

In [7]:

image.shape

(512, 512, 3)